In [1]:
## Shiv, you can ignore this cell as it is only for reading the data, loading the libraries, etc.

# libraries
from dash import Dash, html, dcc, Input, Output
import altair as alt
import pandas as pd
import dash_bootstrap_components as dbc

# Read in data
traits_raw_df = pd.read_csv("../data/breed_traits.csv")
breed_rank_raw_df = pd.read_csv("../data/breed_rank.csv")

# For traits list in checklist input
traits_list_full = traits_raw_df.drop(columns=['Breed' ,'Coat Type', 'Coat Length']).columns

traits_raw_df["BreedID"] = traits_raw_df.index
breed_rank_raw_df["BreedID"] = breed_rank_raw_df.index

/opt/miniconda3/envs/532/lib/python3.10/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
#breed_rank_raw_df[breed_rank_raw_df['2013 Rank'].isnull()] #for debug

In [3]:
# THIS IS THE BEGINNING OF THE CODE I WROTE FOR THIS "SURPRISE ME" FEATURE

surprise_me_raw_df = breed_rank_raw_df.sample()

# The following 3 lines is for avoiding picking a breed that does not have past rank data.
while (surprise_me_raw_df.isnull().any().any()):
    print("something is null.  picking another sample.") #for debug
    surprise_me_raw_df = breed_rank_raw_df.sample()
#surprise_me_raw_df #for debug

In [4]:
# Merge the 2 dataframes together
surprise_me_raw_df = surprise_me_raw_df.merge(traits_raw_df.drop('Breed', axis=1), 
                                              how='left', on='BreedID')
    
# The following few lines of code is for changing the column names to tidy the "Rank year" and "Rank" columns.
col_list = list()
for year in range(2013, 2021):
    new_col_name = str(year)
    col_list.append(str(year))
    old_col_name = new_col_name + " " + "Rank"
    surprise_me_raw_df.rename(columns={old_col_name:new_col_name}, inplace=True)

#surprise_me_raw_df #for debug

In [5]:
# The dataframe now has the columns Breed, BreedID, Image (URL), Rank year and Rank. Please 
surprise_me_df = surprise_me_raw_df.melt(id_vars = ['Breed', 'BreedID', 'Image'], value_vars=col_list, var_name='Rank year', value_name='Rank')
surprise_me_df

,Breed,BreedID,Image,Rank year,Rank
0,Fox Terriers (Wire),103,https://www.akc.org/wp-content/uploads/2017/11...,2013,96.0
1,Fox Terriers (Wire),103,https://www.akc.org/wp-content/uploads/2017/11...,2014,95.0
2,Fox Terriers (Wire),103,https://www.akc.org/wp-content/uploads/2017/11...,2015,94.0
3,Fox Terriers (Wire),103,https://www.akc.org/wp-content/uploads/2017/11...,2016,101.0
4,Fox Terriers (Wire),103,https://www.akc.org/wp-content/uploads/2017/11...,2017,99.0
5,Fox Terriers (Wire),103,https://www.akc.org/wp-content/uploads/2017/11...,2018,101.0
6,Fox Terriers (Wire),103,https://www.akc.org/wp-content/uploads/2017/11...,2019,101.0
7,Fox Terriers (Wire),103,https://www.akc.org/wp-content/uploads/2017/11...,2020,104.0


In [6]:
# This is the ranking trend plot.
surprise_me_plot = alt.Chart(surprise_me_df).mark_line().encode(
    y=alt.Y('Rank:Q', scale=alt.Scale(zero=False, reverse=True)),
    x=alt.X('Rank year:Q', axis=alt.Axis(format='.0f')),
    tooltip=['Breed', 'Rank:Q']
).properties(
    width=400,
    height=200,
    title=f"Surprise! Have you heard of {surprise_me_df.Breed[0]}? Here is its ranking trend."
).interactive()
surprise_me_plot

alt.Chart(...)